# Entrenamiento del modelo


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# Asume que las imágenes son de tamaño 299x299 para Xception
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'directorio_datos'
batch_size = 32

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


In [ ]:
import timm
import torch

model_name = 'xception'  # Asegúrate de que el nombre coincide con los disponibles en timm
model = timm.create_model(model_name, pretrained=True, num_classes=5)

# Asegúrate de adaptar el resto del código para entrenar, evaluar y usar el modelo


In [ ]:
from torch import optim
from torch import nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [ ]:
num_epochs = 12

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / dataset_sizes['train']
    epoch_acc = running_corrects.double() / dataset_sizes['train']

    print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')


# Guarda el modelo

In [ ]:
torch.save(model.state_dict(), 'xception_modell.pth')


# Matriz de confusion y Recall

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import numpy as np
import os
from sklearn.metrics import recall_score

# Asumiendo que 'model' ya está definido y cargado como se mostró anteriormente

# Transformaciones para las imágenes de prueba
transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Cargar las imágenes de prueba
test_dir = 'directorio_datos_test'
test_dataset = datasets.ImageFolder(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4)
class_names = test_dataset.classes

# Función para realizar predicciones y calcular la matriz de confusión
def predict_and_confusion_matrix(model, dataloader):
    y_pred = []
    y_true = []
    model.eval()  # Establecer el modelo en modo de evaluación
    with torch.no_grad():  # No necesitamos calcular gradientes
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    
    # Calcular la matriz de confusión
    conf_mat = confusion_matrix(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='macro')
    return conf_mat,recall

# Calcular y mostrar la matriz de confusión
conf_mat, recall = predict_and_confusion_matrix(model, test_loader)
print(conf_mat)



print(f'Recall: {recall}')
